In [1]:
import fitz
import json
import re
import nltk

PDF_FILE_PATH = "./data/GTM-Construction-Safety-Manual.pdf"
JSON_FILE_PATH = "./data/original_manual.json"

In [124]:
doc = fitz.open(PDF_FILE_PATH)
big_point = -1
middle_point = -1
small_point = -1
save_js = {}
for i, page in enumerate(doc):
    if i < 15 or i > 246:
        continue
    print(f"page {i+1}")
    blocks = page.get_text("dict")["blocks"][6:]
    for block in blocks:
        if "lines" not in block.keys():
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                text = span["text"]
                # Bold 체 중에 선택
                if "Bold" in span["font"]:        
                    # 대제목 가져오기
                    if re.match(r'^\d+\.\s', text.strip()):
                        print("BIG TITLE:", text)
                        big_point += 1
                        save_js[big_point] = {"title_text":text, "text":""}
                        middle_point = 0
                        small_point = 0
                        continue
                    # 중제목 가져오기
                    elif re.match(r'^\d+\.\d+\.\s', text.strip()):
                        print("MIDDLE TITLE:", text)
                        middle_point += 1
                        if "middle" not in save_js[big_point].keys():
                            save_js[big_point]["middle"] = {}
                        save_js[big_point]["middle"][middle_point] = {"title_text":text, "text":""}
                        small_point = 0
                        continue
                    # 소제목 가져오기
                    elif text != " " and text[0] == " " and text[1] != " ":
                        print("SMALL TITLE:", text)
                        small_point += 1
                        if "small" not in save_js[big_point]["middle"][middle_point].keys():
                            save_js[big_point]["middle"][middle_point]["small"] = {}
                        save_js[big_point]["middle"][middle_point]["small"][small_point] = {"title_text":text, "text":""}
                        continue
                # 각 제목의 text
                if middle_point == 0 and small_point == 0:
                    save_js[big_point]["text"] += text
                elif small_point == 0:
                    save_js[big_point]["middle"][middle_point]["text"] += text
                else:
                    save_js[big_point]["middle"][middle_point]["small"][small_point]["text"] += text
                print(big_point, middle_point,  small_point)

page 16
BIG TITLE: 1. Leadership and Governance 
MIDDLE TITLE: 1.1. Supplier Management System 
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
0 1 0
MIDDLE TITLE: 1.2. Leadership in Action 
0 2 0
0 2 0
0 2 0
0 2 0
0 2 0
MIDDLE TITLE: 1.3. Stop Work, Imminent Hazard, and Right to Refuse Unsafe Work 
0 3 0
0 3 0
0 3 0
0 3 0
page 17
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
0 3 0
SMALL TITLE:  Responsibilities 
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
page 18
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3 1
0 3

In [126]:
save_js[12]["middle"]

{1: {'title_text': '13.1. Incident Reporting, Investigation and Learning ',
  'text': 'Suppliers shall ensure that they immediately report to Company all EHS incidents including work related injuries and illnesses, vehicle, property damage, Near Misses, spills and releases, fires, or explosions arising from the Supplier’s execution of work.  Reportable Event: An event is an unplanned occurrence that interrupts a work activity.   Reportable events include:  People:  Injuries or illnesses to Contractors or Visitors  Environment:  Unauthorized or unplanned releases or  Impacts to air, land, water, or wildlife  Property Damage:  Ruptures, fires, or explosions  Motor Vehicle, aircraft, or watercraft incidents involving Contractors  Damage to Company, Supplier, or third-party property or  Theft or vandalism  Regulatory and Other:  Government agency visits or contact  Permit / license contraventions  Unauthorized activity on ROW  Landowner / public complaints or  Security Threats  Near Misses

# 0. 데이터 분석
가장 중요한 내용이 나오는 page 16 이후에 관한 이야기
## 0.1. pdf 겉보기
1. 대제목과 중제목에 붙은 숫자는 text 처리가 되지만, **소제목에 붙은 숫자는 text 처리가 되어 있지 않다**.
2. 대/중/소 제목 모두 대소문자가 섞여 있다.
3. 소제목 바로 다음의 "문장 기호가 붙어 있지 않은 문장"은 긴 문장으로 이루어 지거나(뒤에 맡침표가 있던가) 아니면 ":"이 붙어있다.
## 0.2. text로 추출
1. 문장들이 모두 '\n'으로 나누어져 있다.
2. '\n \n' 이렇게 두 번 오는 경우도 있는 데, 어떤 경우인 지 잘 모르겠다.

# 1. 전처리 과정
## 1.1. 불필요한 기호나 표현 없애기

In [3]:
original_js[203]["text"]

"Construction Safety Manual \nVersion Date: 2023-01-27 | V 2.1  \n \n \n \n© Enbridge Gas Transmission and Midstream   |   Manual \nINTERNAL INFORMATION – Uncontrolled Copy if Printed or Downloaded. \nPage 204 of \n270 \n \n \nBe aware of possible damage to trees and power lines due to ice buildup \n \nAvoid travel in these conditions \n Extreme Heat/Heat Stress: \n \nConsult the climatic condition reports from your local weather service during \nHazard Assessment \n High Wind: \n \nWhen wind conditions exceed 50 km/hr. (30 mph), the Hazard Assessment shall \nbe reviewed and adjusted to take the wind conditions into consideration, or the \nactivity shall be suspended until wind conditions are more favorable \n \nConsider other Hazards such as dust and debris, secure any loose materials \n Working after Sunset and before Sunrise: \n \nWork after dusk shall not be permitted unless the following conditions are met: \n\uf0a7 \nPrior approval shall be obtained from the Company Representativ

In [15]:
[i for i in re.split(r'\n', original_js[203]["text"]) if i != "" and  i[0] == " " ][-30:]

[' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' Extreme Heat/Heat Stress: ',
 ' ',
 ' High Wind: ',
 ' ',
 ' ',
 ' Working after Sunset and before Sunrise: ',
 ' ',
 ' ',
 ' Suppliers ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 ' ',
 '  ']

In [140]:
re.sub(r"[^\w\d\s\.]", "", original_js[1]["text"])

'Construction Safety Manual \nVersion Date 20230127  V 2.1  \n \n \n \n Enbridge Gas Transmission and Midstream      Manual \nINTERNAL INFORMATION  Uncontrolled Copy if Printed or Downloaded. \nPage 2 of 270 \n \nTable of Contents \nTable of Contents \n.............................................................................................................................................................. \n2 \nPurpose \n.............................................................................................................................................................................. \n7 \nScope ................................................................................................................................................................................. \n7 \nExpectations \n...................................................................................................................................................................... \n7 \nSafety Princi

In [155]:
# 문자에 저장된 기호 및 주석 삭제
save_js = {}
for i, page in enumerate(doc):
    text = page.get_text()
    filtered_content = re.sub(r"[^\w\d\s\.]", "", text)
    filtered_content = filtered_content.replace(
        f"Construction Safety Manual \nVersion Date 20230127  V 2.1  \n \n \n \n Enbridge Gas Transmission and Midstream      Manual \nINTERNAL INFORMATION  Uncontrolled Copy if Printed or Downloaded. \nPage {i+1} of 270 \n \n",
        "",
    )
    if filtered_content.split(" ")[0] =r"[0-9].":
        print(i)
    save_js[i] = {"page": (i + 1), "text": filtered_content}

In [259]:
te = "11. Hello"

In [260]:
if re.match(r"[0-9]\.[^\d]", te):
    print("hello")

## 1.2. 대소문자 통일하기

In [142]:
for key, value in save_js.items():
    save_js[key] = {"page": (i + 1), "text": value["text"].lower()}

## 1.3. 토큰 단위로 분할하기
### 1.3.1 단어 단위로 분할하기

In [143]:
# 상대적으로 최근에 nltk에서 제공하는 토크나이저. 영어 외 프랑스어나 독일어 등에서 좋은 성능을 낸다.
from nltk.tokenize.toktok import ToktokTokenizer
tok = ToktokTokenizer()

In [144]:
print(tok.tokenize(save_js[7]["text"]))

['safety', 'principals', 'figure', '1', 'path', 'to', 'zero']


### 1.3.2 문장 단위로 토큰화하기

#### 고민거리
- 상황: 주어진 text에 제목들이 포함되어 있으며, 그 제목을 구분하는 방식이 \n 이다. 또 내용 문장이 줄 바꿈 될 때에 연속되지 않고 모두 \n으로 구분되어 있다.
- 문제: \n을 기준으로 구분을 하면 제목과 내용을 구분할 수 있지만, 하나의 문장이 여러 문장으로 구분이 된다. \n이 아닌 '.', '?', '!'으로 구분을 하면 내용을 구분할 수 있지만, 제목을 내용과 구분할 수 없다.
- 해결책: ?
#### 특이 사항
위에 이미 "." 같은 기호들을 모두 제거해서 문장 부호 단위로 자를 순 없음. 그래서 지금은 \n으로 자르기로 결정

In [145]:
# \n 단위로 자르고 빈 문자열은 제거
for key, value in save_js.items():
    save_js[key]["sequence_token"] = [i.rstrip() for i in re.split(r'\n', value["text"]) if i.rstrip() != "" ]

In [149]:
save_js[54]["sequence_token"]

['task',
 'bond from',
 'bond to',
 'notes',
 'hydrovacing near',
 'underground electrical',
 'wires',
 'wandgun',
 'dig tube',
 'mat  1',
 'mat 1',
 'grounding mat  1',
 'grounding mat  2',
 'mat  2',
 'hydrovac truck',
 'for distances greater than  1.8 m',
 '6 ft. from the hydrovac truck it',
 'may not be necessary to bond the',
 'mat to the truck step 4.',
 'note  some tasks require more than one bonding cable. the numbers listed in this table represent steps to be',
 'taken for proper bonding i.e. bond 1 to 1 and 2 to 2 etc..',
 '6.7. brush cutting clearing and vegetation management',
 ' suppliers',
 'suppliers shall ensure that',
 'the terrain is surveyed in advance of initiating work for hazards prior to operations.',
 'all applicable precautions are implemented when overhead clearing of vegetation.',
 'if clearing is required within limits of approach boundary to overhead powerlines a certified',
 'and  or licensed professional arborist will be utilized to perform the work.  ref

## 1.4 단어의 품사 찾기
#### 문제거리
여기서 문장의 제목/중제목/소제목을 나눠 보려 했지만, 6. 이나 6.1.은 text로 추출이 되지만, 문제는 6.1.1.은 text가 아니다. 그래서 위 문장 토큰화 부분에서 다르게 처리를 해야 한다. 이제 이 방식을 찾아봐야 한다.

## 1.5. 불용어 제거
제거하면 좋지만, 아직은 그대로 놔두도록 하겠습니다.

In [132]:
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')

In [138]:
"couldn" in stopwords_list

True